In [22]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.exceptions import UndefinedMetricWarning
import warnings
import matplotlib.pyplot as plt
import random
from imblearn.under_sampling import RandomUnderSampler
import math
import os
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [23]:
# Definição de variáveis de aplicação geral
divisao_cjt_teste = 0.2
caminho_graficos = r'C:\Users\alexa\OneDrive\Documentos\TCC MBA\Etapa final TCC\Ambiente de testes\Testes consolidados\Gráficos Kfold'
caminho_base = r'C:\Users\alexa\OneDrive\Documentos\TCC MBA\Etapa final TCC\Ambiente de testes\Dados.xlsx'
caminho_pasta_atual = r'C:\Users\alexa\OneDrive\Documentos\TCC MBA\Etapa final TCC\Ambiente de testes\Testes consolidados'
caminho_modelos = r'C:\Users\alexa\OneDrive\Documentos\TCC MBA\Etapa final TCC\Ambiente de testes\Testes consolidados\Métricas.xlsx'
planilha_modelos = 'Modelos kfold'
planilha_original = 'DadosOriginaisRedux'
planilha_oversample = 'DadosOsRedux'
planilha_smotex2 = 'DadosSMOTE2'
planilha_smote = 'DadosSmoteRedux'
output = 'output-kfold.txt'
# Definição das seeds
seed_value = 1
random.seed(seed_value)         # Python random
np.random.seed(seed_value)      # Numpy
tf.random.set_seed(seed_value)  # TensorFlow/Keras

In [24]:
# Importação da base de dados
modelos = pd.read_excel(io=caminho_modelos, sheet_name=planilha_modelos)

In [25]:
# Definição da função para tratamento dos dados. Retorna os dados já divididos em treinamento e teste
def tratamento_dados(caminho_arquivo:str, nome_planilha:str, aplica_undersample:bool):
    
    # Importação dos dados para um DataFrame pandas
    dados = pd.read_excel(io=caminho_arquivo, sheet_name=nome_planilha)

    # Excluindo o campo Data, visto que o ano em si não é importante na análise, apenas a janela temporal
    try:
        dados.drop(['Data'], axis=1, inplace=True)
    except:
        pass

    dados_tratamento = None

    if aplica_undersample:

        # Aplicando a técnica de undersampling para diminuir a quantidade da categoria predominante (RJ = 0) e balancear os dados
        # Filtrando os dados e pegando apenas 1 registro de cada empresa, do ano 5, que contém de fato 0 e 1
        dados_undersampling = dados[dados['Ano'] == 5][['Name', 'RJ']]

        #Transformando em um array de 2 dimensões para ser utilizado com a classe RandomUnderSampler, e separando em x e y
        x_undersampling = dados_undersampling['Name'].values.reshape(-1, 1)
        y_undersampling = dados_undersampling['RJ']

        # Construindo o objeto para aplicar a reamostragem, com fator de 30% da classe menor em relação à maior
        rus = RandomUnderSampler(sampling_strategy=0.3, random_state=42)

        # Aplicando a reamostragem
        x_resampled, _ = rus.fit_resample(x_undersampling, y_undersampling)

        # Transformando o array x, que contém o nome das empresas mantidas após RUS em um array de uma dimensão
        empresas_remanescentes = x_resampled.flatten()

        # Filtrando os dados originais e mantendo apenas os nomes das empresas que saíram do resultado de RUS
        dados_tratamento = dados[dados['Name'].isin(empresas_remanescentes)]


    else:

        # Caso RUS não seja ativado na chamada da função, simplesmente ignora o processo
        dados_tratamento = dados
 

    # Inicializando uma variável de lista vazia para conter os dados organizados para a análise
    dados_organizados = []

    # Criando uma variável com os nomes (códigos) distintos de todas empresas
    empresas_unicas = dados_tratamento['Name'].unique()

    # Iterando sobre os nomes distintos das empresas e separando os dados dos indicadores com o nome de cada empresa em uma lista
    for empresa in empresas_unicas:
        dados_empresa = dados_tratamento[dados_tratamento['Name'] == empresa]
        dados_empresa = dados_empresa.iloc[:,0:-2].values
        dados_organizados.append(dados_empresa)
    
    # Transformando a lista em um array numpy
    dados_organizados = np.array(dados_organizados)

    # Atribuindo o array à variável X, que representa as variáveis independentes do modelo
    X = dados_organizados

    # Atribuindo à variável y, que represente a variável dependente, os valores da variável dependente RJ
    y = dados_tratamento[dados_tratamento['Ano'] == 5]['RJ']

    return train_test_split(X, y, test_size=divisao_cjt_teste, random_state=42)

In [26]:
# Define a função que constrói o modelo de rede neural, retornando-a ao fim
def return_model(input_shape, lstm:int, lstm2:str, dense2:str, dropout_rate:float, learning_rate:float, ativacao:str):

    # Define um modelo sequencial LSTM
    model = Sequential()

    # Adiciona uma camada lstm
    model.add(LSTM(lstm, input_shape=input_shape, activation=ativacao, return_sequences=True if lstm2 == 's' else False))

    # Adiciona dropout
    model.add(Dropout(dropout_rate))

    # Camadas adicionais condicionais
    if lstm2 == 's':
        model.add(LSTM(lstm, activation=ativacao, return_sequences=False))
    if dense2 == 's':
        model.add(Dropout(dropout_rate))
        model.add(Dense(32, activation=ativacao))

    # Adiciona a camada densa de saída
    model.add(Dense(1, activation='sigmoid'))

    # Define o otimizador
    optimizer = Adam(learning_rate=learning_rate)

    # Compila o modelo
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[BinaryAccuracy()])

    return model

In [27]:
# Define uma função para normalização dos dados
def normalizacao(caminho_base_, planilha_, undersample_):

    # Atriubui valores às variáveis independentes (x) e dependentes (y) de treinamento e teste, utilizando função previamente definida
    X_train, X_test, y_train, y_test = tratamento_dados(caminho_base_, planilha_, undersample_)

    # Separa as quantidades de empresas para teste e validação, utilizada a seguir para remodelar uma matriz
    qtd_empresas_teste = y_test.shape[0]
    qtd_empresas_treino = y_train.shape[0]

    # Separando o nome das empresas de treino em uma variável exclusiva, para posterior identificação caso necessário
    empresas_treino = X_train[:,:,0:1]

    # Remodela empresas treino para um array de uma dimensão
    empresas_treino = empresas_treino.reshape(qtd_empresas_treino, 5)

    empresas_treino = empresas_treino[:,0]

    # Separando os dados quantitativos (indicadores) e redefinindo a variável X_train com os mesmos
    X_train = X_train[:,:,1:5]

    # Definindo a variável n_features com base no número de indicadores atual do estudo
    _,_,n_features = X_train.shape

    # Realizando os mesmos procedimentos nas empresas de teste
    empresas_teste = X_test[:,:,0:1]
    empresas_teste = empresas_teste.reshape(qtd_empresas_teste, 5)
    empresas_teste = empresas_teste[:,0]
    X_test = X_test[:,:,1:5]

    # Inicializando uma lista vazia onde serão armazenados os scalers, os objetos que farão a normalização dos dados de teste e treino
    scalers = []

    # Inicializa arrays para guardar os dados normalizados
    X_train_normalized = np.empty(X_train.shape)
    X_test_normalized = np.empty(X_test.shape)


    # Iteração partindo de zero até n_features vezes
    for i in range(n_features):

        # Inicializa o objeto scaler a ser utilizado nesta iteração/indicador
        scaler = StandardScaler()

        '''
            Aplica o método fit_transform sobre o i-ésimo indicador - este método calcula média e desvpad,
            subtraindo a média dos valores e dividindo esse resultado pelo desvpad para normalizá-los.
        '''
         
        X_train_normalized[:, :, i] = scaler.fit_transform(X_train[:, :, i])

        # O método transform dessa vez é aplicado sobre a base de teste, utilizando o mesmo desvpad e média da base de treino
        X_test_normalized[:, :, i] = scaler.transform(X_test[:, :, i])
        
        # Armazena o objeto scaler para posteridade, caso se deseje desnormalizar os dados
        scalers.append(scaler)
    return X_train_normalized, X_test_normalized, y_train, y_test

In [28]:
# Define a função que realizará a validação propriamente dita
def kfold_custom(pesos, lstm, lstm2, dense2, tx_dropout, tx_aprendizagem, tamanho_lote, epocas, ativacao, caminho_base_, planilha_, undersample_, n_splits_, alias, trial, ajuste_epocas):

        # Inicializa variáveis de lista para receberem os resultados das métricas de avaliação
        results_acc = []
        results_prec = []
        results_rec = []
        results_f1 = []
        results_auc = []

        # Aplica normalização nos dados da base        
        X_train_normalized, X_test_normalized, y_train, y_test = normalizacao(caminho_base_, planilha_, undersample_)

        # Extrai a janela temporal e número de indicadores do formato da matriz/array X_train_normalized        
        _, anos, indicadores = X_train_normalized.shape

        # Define a variável que será utilizada no formato de entrada dos dados na arquitetura do modelo
        input_shape = (anos, indicadores)

        # Inicializa um objeto da classe KFold para realizar as divisões dos dados em cada dobra        
        kfold = KFold(n_splits=n_splits_, shuffle=True, random_state=42)

        # Transforma y_train em array
        y_train_array = y_train.values

        # Variável que determina quantas linhas de gráficos haverão na figura gerada para cada modelo
        linhas = math.ceil(n_splits_ / 2)

        # Constrói o objeto onde os gráficos serão plotados
        fig, axs = plt.subplots(linhas, 2, figsize=(15, 5 * linhas))
        fig.suptitle('Perda de Treino vs. Validação por Dobra do KFold')

        # Inicia iteração sobre cada dobra gerada pelo objeto kfold com base nos dados de treino normalizados
        fold_num = 1
        for i, (train_index, test_index) in enumerate(kfold.split(X_train_normalized)):
                
                # Dividindo os dados de treino e teste da dobra, com base na parcela de dados da divisão de treino geral
                X_train_fold = X_train_normalized[train_index]
                X_test_fold = X_train_normalized[test_index]
                y_train_fold = y_train_array[train_index]
                y_test_fold = y_train_array[test_index]

                # Retorna o modelo que será utilizado
                model = return_model(input_shape, lstm, lstm2, dense2, tx_dropout, tx_aprendizagem, ativacao)

                # Se o argumento pesos for verdadeiro, cria o objeto class_weight_dict
                if pesos:

                        classes = np.unique(y_train)
                        class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
                        class_weight_dict = dict(zip(classes, class_weights))

                '''
                Verifica se a variável ajuste_epocas já é verdadeira. 
                Isto é necessária para que todas dobras de um modelo sejam treinadas com o mesmo número de épocas, 
                devido ao early stopping que pode ser aplicado de forma diferente a cada iteração
                '''
                if not ajuste_epocas:

                        early_stopping = EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                restore_best_weights=True)
                        
                        # Executa o treinamento do modelo
                        history = model.fit(
                                X_train_fold, y_train_fold, epochs=epocas, batch_size=tamanho_lote, 
                                validation_split=divisao_cjt_teste, verbose=0, callbacks=[early_stopping],
                                class_weight=class_weight_dict if pesos else None
                                )
                else:
                        # Executa o treinamento do modelo
                        history = model.fit(
                                X_train_fold, y_train_fold, epochs=epocas, batch_size=tamanho_lote, 
                                validation_split=divisao_cjt_teste, verbose=0,
                                class_weight=class_weight_dict if pesos else None
                                )    

                # Separa o histórico da curva de perda no treino e validação     
                train_losses = history.history['loss']
                val_losses = history.history['val_loss']

                '''
                Realiza as previsões no conjunto de teste da dobra, transformando o resultado para binário.
                Necessário para que tanto o Y predito quanto o real correspondam ao mesmo tipo de dado
                '''
                y_pred = model.predict(X_test_fold).flatten()
                y_pred_binario = (y_pred > 0.5).astype(int)

                # Ajusta o numero de épocas na primeira iteração, conforme early stopping
                if not ajuste_epocas:
                       epocas = len(history.history['loss'])
                       ajuste_epocas = True

                # Cálculo das métricas
                acc = round(accuracy_score(y_test_fold, y_pred_binario), 4)
                prec = round(precision_score(y_test_fold, y_pred_binario), 4)
                rec = round(recall_score(y_test_fold, y_pred_binario), 4)
                f1 = round(f1_score(y_test_fold, y_pred_binario), 4)
                auc_roc = round(roc_auc_score(y_test_fold, y_pred_binario), 4)

                # Atualização da variável de lista conforme métricas da dobra
                results_acc.append(acc)
                results_prec.append(prec)
                results_rec.append(rec)
                results_f1.append(f1)
                results_auc.append(auc_roc)

                # Print no console para acompanhamento
                print(f"Dobra {fold_num}")
                print("Accuracy:", acc)
                print("Precision:", prec)
                print("Recall:", rec)
                print("F1 Score:", f1)
                print("ROC AUC:", auc_roc)

                # Calcula qual a próxima posição (linha, coluna) será plotado gráfico na figura criada anteriormente
                linha = 0 if i < 1 else linha + 1 if i % 2 == 0 else linha
                coluna = 1 if i % 2 > 0 else 0

                # Criando e plotando o gráfico da dobra atual no conjunto de gráficos do modelo
                ax = axs[linha, coluna]
                ax.plot(train_losses, label='Perda de Treino')
                ax.plot(val_losses, label='Perda de Validação')
                ax.set_title(f'Dobra {fold_num}')
                ax.set_xlabel('Épocas')
                ax.set_ylabel('Perda')
                ax.legend()
                
                # Variável de controle das dobras
                fold_num += 1
                
        # Configura e salva os gráficos do modelo atual
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.ylim(bottom=0, top=1)
        plt.savefig(os.path.join(caminho_graficos, f'modelo_{trial}_{alias}.png'))
        plt.close()
        metricas = f'Alias: {alias}, Trial: {trial}, Acc: {results_acc}, Prec: {results_prec}, Rec: {results_rec}, F1: {results_f1}, AUC_ROC: {results_auc}, Épocas ajustadas: {epocas}'

        # Salva os dados das métricas no arquivo de texto
        with open(os.path.join(caminho_pasta_atual, output), 'a') as f:
            print(metricas, file=f)

In [ ]:
'''
    Itera sobre as linhas da tabela Pandas contendo os dados com os parâmetros dos modelos selecionados para a validação cruzada,
    construindo-os e inicializando todo o processo
'''
for index, row in modelos.iterrows():
    alias = row['Alias']
    trial = row['Trial']
    lstm = row['lstm']
    lstm2 = row['lstm2']
    dense2 = row['dense2']
    ativacao = row['ativacao']
    dropout_rate = row['dropout']
    learning_rate = row['learning_rate']
    batch_size = row['batch_size']
    epochs = row['epochs']

    print(f'Calculando {alias}-{trial}, índice {index} de {modelos.shape[0]}')

    # Chama a função principal, passando variáveis conforme cada estratégia definida
    kfold_custom(
        pesos=True if 'P' in alias else False, 
        lstm=lstm, 
        lstm2=0 if lstm2 == 'n' else lstm, 
        dense2=0 if dense2 == 'n' else 32, 
        tx_dropout=dropout_rate, 
        tx_aprendizagem=learning_rate, 
        tamanho_lote=batch_size, 
        epocas=epochs, 
        ativacao=ativacao, 
        caminho_base_=caminho_base, 
        planilha_=planilha_oversample if 'OS' in alias else planilha_smote if 'SMOTE' in alias else planilha_original, 
        undersample_=True if 'US' in alias else False,
        n_splits_ = 5,
        alias = alias,
        trial = trial,
        ajuste_epocas = False
    )